# Data-Pre-Processing for Training Acceleration

This notebook splits the original dataset into 4 to process the original raw data into the working volumes and builds a zip file which will later be 1/4 of the preprocessed volumes dataset. This approach was suggested by TA's and discussed on Piazza.

INPUT: Needs the whole dataset's series folder and the train.csv file. It uses the SIDs of the patients to preprocess the volume (depending on the modality) and save the volume with the same name as the original folder that contained the dicom files of said patient with the same sid number

# Initialization

In [1]:
#Libraries from Demo
import os
import shutil
from collections import defaultdict

import pandas as pd, ast
import polars as pl
import pydicom as dicom

import kaggle_evaluation.rsna_inference_server

#Libraries from attempt
import torch
import random
from torch.utils.data import Dataset, DataLoader

#For visalization
import matplotlib.pyplot as plt

#For ROI Extraction:
import numpy as np
from scipy.ndimage import zoom as ndi_zoom

#For Running Pre-Processing
import os, numpy as np, pandas as pd, multiprocessing as mp
from tqdm.auto import tqdm

ID_COL = 'SeriesInstanceUID'

LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

# All tags (other than PixelData and SeriesInstanceUID) that may be in a test set dcm file
DICOM_TAG_ALLOWLIST = [
    'BitsAllocated',
    'BitsStored',
    'Columns',
    'FrameOfReferenceUID',
    'HighBit',
    'ImageOrientationPatient',
    'ImagePositionPatient',
    'InstanceNumber',
    'Modality',
    'PatientID',
    'PhotometricInterpretation',
    'PixelRepresentation',
    'PixelSpacing',
    'PlanarConfiguration',
    'RescaleIntercept',
    'RescaleSlope',
    'RescaleType',
    'Rows',
    'SOPClassUID',
    'SOPInstanceUID',
    'SamplesPerPixel',
    'SliceThickness',
    'SpacingBetweenSlices',
    'StudyInstanceUID',
    'TransferSyntaxUID',
]

In [2]:
def seed_everything(seed=42):
    """
    Set random seeds for reproducibility in deep learning projects.
    
    Args:
        seed (int): Random seed value (default: 42)
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything()

In [3]:
# training data's labels
train_csv = "/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv"
#training data's localizable labels
localizer_csv = "/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv"
#Path to the folder "series, contains all the patients"
DICOM_PATH = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"
#Path to a patient's DICOM folder
dicom_dir = "/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647"
#Path to a patient's NII file (contains all slices already)
nift_path = "/kaggle/input/rsna-intracranial-aneurysm-detection/segmentations"
#Path to one slice DICOM file from one patient
dicom_sample = "/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10004044428023505108375152878107656647"

In [4]:
# Model configuration
TARGET_SIZE = (160, 160, 160)  # Reduced size for memory efficiency
TARGET_ISO_BRAIN = 1.25
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_df = pd.read_csv(train_csv)

# Supporting Functions


## Load the Localizers table with better preview (e.g., split coordinates into columns)

In [5]:
def _load_localizers_csv(path):
    """
    INPUT: 
        path: path to the localizers csv file.
    OUPTUT:
        df: localizer table with updated columns, the lession localization was splitted into 2 columns.
    """
    df = pd.read_csv(path)
    # columns: SeriesInstanceUID, SOPInstanceUID, coordinates, location
    df["coords"] = df["coordinates"].apply(lambda s: ast.literal_eval(s) if isinstance(s,str) else s)
    df["x"] = df["coords"].apply(lambda d: float(d["x"]))
    df["y"] = df["coords"].apply(lambda d: float(d["y"]))
    return df[["SeriesInstanceUID","SOPInstanceUID","x","y","location"]]


loc_df = _load_localizers_csv(localizer_csv)

## Full Volume Pre-Processing

In [6]:
def _get_slice_order(slice_sample):
    """
    INPUT: 
    slice_sample: dicom read single slice.
    OUPTUT:
    z_cord: slice's Z axis index for ordering.
    """
    #check if the slice/dicom_file has ImageOrientationPatient
    iop = getattr(slice_sample, "ImageOrientationPatient", None)
    ipp = getattr(slice_sample, "ImagePositionPatient", None)

    if iop is not None and ipp is not None and len(iop)>=6 and len(ipp)>=3:
        x_cord = np.array(iop[:3], float)      # rows dir (x)
        y_cord = np.array(iop[3:6], float)     # cols dir (y)
        z_cord = np.cross(x_cord, y_cord)      # slice normal (z)
        ipp = np.array(ipp[:3], float)
        return float(np.dot(ipp, z_cord))

    ins_num = getattr(slice_sample, "InstanceNumber", None)
    if ins_num is not None:
        z_cord = int(ins_num)
        return z_cord 

def _slice_by_slice_correct_rescale(list_of_slices):
    """
    INPUT: 
    list_of_slices: List of dicom read slices (order does not matter).
    OUPTUT:
    corrected_slices: 1 channel, propper gray scale and corrected list of dicom slices in Array form.
    """
    corrected_slices = []
    for slce in list_of_slices:
        slce_array = slce.pixel_array.astype(np.float32)
        
        #Fix the contrast of some images in case the smaller values are white instead of black for consistency
        # https://dicom.innolitics.com/ciods/rt-dose/image-pixel/00280004
        # https://dgobbi.github.io/vtk-dicom/doc/api/imageDisplay.html?utm_source=chatgpt.com
        phot = str(getattr(slce, "PhotometricInterpretation", "MONOCHROME2")).upper()
        if phot == "MONOCHROME1":
            maxv = (2**int(getattr(slce, "BitsStored", arr.dtype.itemsize*8))) - 1
            slce_array = maxv - slce_array

        # https://collectiveminds.health/articles/the-ultimate-guide-to-preprocessing-medical-images-techniques-tools-and-best-practices-for-enhanced-diagnosis
        #Collapse channels if there are more than 1, maybe just CT, but lets be sure.
        spp = int(getattr(slce, "SamplesPerPixel", 1))
        if spp == 3 and slce_array.ndim == 3:
            slce_array = slce_array.mean(axis=-1)
        
        # Apply DICOM linear HU rescale because we are using different modalities: https://www.mathworks.com/help/medical-imaging/ug/overview-medical-image-preprocessing.html 
        slope = float(getattr(slce, "RescaleSlope", 1.0))
        inter = float(getattr(slce, "RescaleIntercept", 0.0))
        corrected_slices.append(slce_array * slope + inter)
    return corrected_slices

def _get_slices_spacing(list_of_slices):
    """
    INPUT: 
    list_of_slices: List of dicom read slices (order does not matter).
    OUPTUT:
    spacing_zyx: the pixel spacing from the dicom files in Array form
    """
    sy, sx = map(float, getattr(list_of_slices[0], "PixelSpacing", [1.0,1.0]))
    sz = None
    
    spacing = (sz, sy, sx)
    st = getattr(list_of_slices[0], "SliceThickness", None)
    sbs = getattr(list_of_slices[0], "SpacingBetweenSlices", None)
    if st is not None:
        sz = float(st)
    if sbs is not None:
        sz = float(sbs)

    if sz is None:
        iop = getattr(dicom_slices[0], "ImageOrientationPatient", None)
        if iop is not None and len(iop) >= 6:
            row = np.array(iop[:3], dtype=float)
            col = np.array(iop[3:6], dtype=float)
            norm   = np.cross(row, col)
            scalars = []
            for slc in list_of_slices:
                ipp = getattr(slc, "ImagePositionPatient", None)
                if ipp is not None and len(ipp)>=3:
                    scalars.append(np.dot(np.array(ipp[:3], float), norm))
            if len(scaalrs)>=2:
                diffs = np.diff(np.sort(scalars))
                sz = float(np.median(np.abs(diffs)))
        if sz is None:
            sz = 1.0
    spacing_zyx = (sz, sy, sx)
    return spacing_zyx

def load_dicom_series(dicom_patient_dir):
    """
    INPUT: 
    dicom_patient_dir: Patient folder directory with dicom files per slice.
    OUPTUT:
    dicom_vol: Ordered and rescaled read dicom slices stacked to make a volume
    modality: MRI, CTA, etc....

    Variables Cheat Sheet
    list_slices_files: The list of PATHS for each slice in the patient folder.
    dicom_slices: dicom read files with data.
    """
    list_slices_files = [
        os.path.join(dicom_patient_dir, f) 
        for f in os.listdir(dicom_patient_dir) 
        if f.lower().endswith(".dcm")]
    
    if not list_slices_files: 
        raise ValueError(f"No DICOM slice files found in patient folder: {dicom_patient_dir}.")
    
    dicom_slices = []
    for slice_file in list_slices_files:
        dicom_slice = dicom.dcmread(
            slice_file, stop_before_pixels=False, force=True)
        dicom_slices.append(dicom_slice)

    dicom_slices.sort(key=_get_slice_order)
    modality = getattr(dicom_slices[0], 'Modality')
    the_series_uid = getattr(dicom_slices[0], "SeriesInstanceUID")

    # Calculate the Spacing using the dicom metadata of one slice.
    spacing_zyx = _get_slices_spacing(dicom_slices)

    
    slices = _slice_by_slice_correct_rescale(dicom_slices)
    dicom_vol = np.stack(slices, axis=0)

    return modality, spacing_zyx, dicom_vol, the_series_uid, dicom_slices



def resample_isotropic(vol_zyx, spacing_zyx, target_iso=1, order=1):
    """
    Resample a [Z,H,W] volume to isotropic spacing (target_iso in mm).
    INPUT:
    vol_zyx: np.ndarray [Z,H,W]
    spacing_zyx: tuple (sz, sy, sx) in mm
    target_iso: desired isotropic spacing in mm 
    order: interpolation order (1 = linear)
    OUPTUT:
    vol_iso: np.ndarray resampled volume
    new_spacing_zyx: (target_iso, target_iso, target_iso)
    """
    sz, sy, sx = map(float, spacing_zyx)
    tz = ty = tx = float(target_iso)

    # zoom factors are (old_spacing / new_spacing)
    zoom_factors = (sz / tz, sy / ty, sx / tx)
    vol_iso = ndi_zoom(vol_zyx, zoom=zoom_factors, order=order)
    spacing_iso_zyx = (tz, ty, tx)
    return spacing_iso_zyx, vol_iso.astype(np.float32)

## Modality Unification Pre-Processing

In [7]:
def modality_specific_preprocessing(vol, modality, cta_startegy="clip"):
    """
    Clip the extremes values of a volume (HU scaled)
    INPUT:
    volume: full sampled volume
    OUPTUT: 
    volume: extremes clipped volume
    Variables Cheat Sheet:
    cta_startegy: clip or hu_window
    """
    if modality == 'CTA' or modality == 'CT':
        if cta_startegy == 'clip':
            vol = _clip_by_percentile_per_volume(vol)
        elif cta_startegy == 'hu_window':
            vol = _apply_hu_window(vol)
        elif cta_startegy == 'hu_range':
            vol = _apply_hu_range(vol)
        else:
            vol = _clip_by_percentile_per_volume(vol)
        
        vol = _apply_normalization(vol, norm='min-max')
        return vol
    else:
        vol = _apply_normalization(vol, norm='z-score')
        return vol

def _clip_by_percentile_per_volume(vol, low_p=0.5, high_p=99.5):
    """
    Clip the HU value of the CT volume
    INPUT: 
    vol: volume tensor
    low: lower percentile value to clip
    high: higher percentile value to clip
    
    OUPTUT:
        vol: clipped vol
    """
    lo = np.percentile(vol, low_p)
    hi = np.percentile(vol, high_p)
    vol = np.clip(vol, lo, hi)
    return ((vol - lo) / (hi - lo + 1e-6)).astype(np.float32)

def _apply_hu_window(vol, center=125, width=550):
    """
    INPUT: 
    vol: volume tensor
    center: center of the HU window
    width: width of the HU unit window [center-width, center+width]
    
    OUPTUT:
    vol: volume scaled to the defined window.
    """
    lo = center - width/2.0
    hi = center + width/2.0
    vol = np.clip(vol, lo, hi)
    return ((vol - lo) / (hi - lo + 1e-6)).astype(np.float32)

def _apply_hu_range(vol, hounsfield_range=(100, 600)):
    """
    INPUT: 
    vol: volume tensor
    center: center of the HU window
    width: width of the HU unit window [center-width, center+width]
    
    OUPTUT:
    range_vol: a mask of only the boders
    """
    mask = (vol >= hounsfield_range[0]) & (vol <= hounsfield_range[1])
    range_vol = np.where(mask, vol, 0)
    return range_vol

def _apply_normalization(vol, norm='z-score'):
    """
    INPUT: 
    vol: volume tenso
    norm: normalization strategy (max-min for CTA, z-score for MRI)
    
    OUPTUT:
    vol: Normalized volume tensor.
    """
    if norm == 'min-max':
        return (vol - vol.min()) / (vol.max() - vol.min() + 1e-8)
    elif norm == 'z-score':
        return (vol - vol.mean()) / vol.std()

## Mask processing

In [8]:
def _voxel_size_from_mm(size_mm, spacing_zyx):
    """
    INPUT: 
    size_mm: desired mm size of the volume surrounding the lession
    spacing_zyx: full volume array's spacing
    
    OUPTUT:
    Dz, Dy, Dx = volume dimensions to take from the center of the zoomed in crop.
    """
    Dz_mm, Dy_mm, Dx_mm = size_mm
    sz, sy, sx = spacing_zyx
    Dz = max(1, int(round(Dz_mm / sz)))
    Dy = max(1, int(round(Dy_mm / sy)))
    Dx = max(1, int(round(Dx_mm / sx)))
    return Dz, Dy, Dx

def _find_slice_index_by_sop(slices, slice_id):
    """
    INPUT: 
    slices: ordered list of dicom slices (no arrays, they ahve to be dicom files)
    slice_id: id of the slice with lession, taken from the localizer's csv dataframe's SOPInstanceUID column

    OUPTUT:
    idx from the dicom slices list which has the lession or None if its a healthy patient and its not listed on localizers csv
    """
    for idx, slc in enumerate(slices):
        if getattr(slc, "SOPInstanceUID", None) == slice_id:
            return idx
    return None

def _crop_with_pad(vol, z0, y0, x0, Dz, Dy, Dx, pad_mode="edge"):
    """
    crop the image surrounding the lession and with the desired size. If the region will escape from the volume, pad by repeating the edges of the volume with data.
    
    INPUT: 
    vol: volume arrat
    z0, x0, y0: volume's center coordinate.
    Dz, Dy, Dx:  The added dimensions over the center to get the volume surrounding the lession.
    pad_mode: for the np.pad function
        
    
    OUPTUT:
    idx from the dicom slices list which has the lession or None if its a healthy patient and its not listed on localizers csv
    
    Variables Cheat Sheet:
     z1, y1, z1 = Volume to add in the three dimensions from the center z0, x0, y0
    """
    Z,H,W = vol.shape
    z1 = z0 + Dz
    y1 = y0 + Dy
    x1 = x0 + Dx

    pb = (max(0,-z0), max(0,-y0), max(0,-x0))
    pa = (max(0,z1-Z), max(0,y1-H), max(0,x1-W))
    if any(pb) or any(pa):
        vol = np.pad(vol, ((pb[0],pa[0]),(pb[1],pa[1]),(pb[2],pa[2])), mode=pad_mode)
        z0 += pb[0]; y0 += pb[1]; x0 += pb[2]
        z1 += pb[0]; y1 += pb[1]; x1 += pb[2]
    return vol[z0:z1, y0:y1, x0:x1]

def _center_to_start(cz, cy, cx, Dz, Dy, Dx):
    """
    Get the corner of the positive part in the mask in reference to the center of the image
    INPUT: 
        cz, cy, cx: the original center of our zoomed in volume, AKA lesion location.
        Dz, Dy, Dx: The dimension of the zoomed in volume with respect to the corner.
    
    OUPTUT:
        cornerz, cornery, cornerx: zoom volume's coorinates
    """
    cornerz = int(round(cz - Dz//2))
    cornery = int(round(cy - Dy/2))
    cornerx = int(round(cx - Dx/2))
    return cornerz, cornery, cornerx

In [9]:
def lesion_mask_full_volume(
    vol,
    list_slices_raw,
    spacing_zyx_raw,
    spacing_zyx_iso,
    lession_df,
    series_uid,
    size_mm=(90, 90, 90),
):
    """
    Build a full-volume binary mask based on lesion coordinates.
    
    INPUT
    vol:            3D numpy array (Z_iso, H_iso, W_iso), already resampled to spacing_zyx_iso *1.2
    list_slices_raw: list of original DICOM slices (raw space)
    spacing_zyx_raw: (sz, sy, sx) in mm, from raw DICOM
    spacing_zyx_iso: (tz, ty, tx) in mm, for the iso-resampled volume 'vol'
    lession_df:     dataframe with columns ["SeriesInstanceUID","SOPInstanceUID","x","y",...]
    series_uid:     SeriesInstanceUID for this volume/patient
    size_mm:        physical ROI size (Dz_mm, Dy_mm, Dx_mm) around lesion center, default as per top solution 4 from competition

    Returns:
        mask same shape as vol, 1 in ROI cube, 0 elsewhere
        info (dict):  metadata for sanity checks - Used ONLY FOR DEBUGGING and personal references/testing
    """
    #Get the shape of the volume, which will be resued
    Z_iso, H_iso, W_iso = vol.shape

    # raw and iso spacings
    sz, sy, sx = map(float, spacing_zyx_raw)
    tz, ty, tx = map(float, spacing_zyx_iso)
    kz, ky, kx = (sz / tz, sy / ty, sx / tx)  # raw -> iso scaling factors

    # raw dimensions from DICOM slices/instances
    raw_H = int(getattr(list_slices_raw[0], "Rows", 0))
    raw_W = int(getattr(list_slices_raw[0], "Columns", 0))
    raw_Z = len(list_slices_raw)

    # look for the patient in the lessions table/train_localisations.csv
    rows = lession_df[lession_df["SeriesInstanceUID"] == series_uid]
    has_lesion = len(rows) > 0
    cz = cy = cx = None

    if has_lesion: #If patient has lession then we create a vinary volume if not its all zeroes
        row = rows.iloc[0]
        sop = str(row["SOPInstanceUID"])
        cz_raw = _find_slice_index_by_sop(list_slices_raw, sop)

        if cz_raw is None:
            has_lesion = False
        else:
            # raw x,y in pixel coordinates from localizer CSV
            cx_raw = float(row["x"])
            cy_raw = float(row["y"])

            # clamp in raw grid
            cx_raw = max(0, min(raw_W - 1, cx_raw))
            cy_raw = max(0, min(raw_H - 1, cy_raw))

            # map to iso grid
            cz = int(round(cz_raw * kz))
            cy = int(round(cy_raw * ky))
            cx = int(round(cx_raw * kx))

            # clamp in iso grid
            cz = max(0, min(Z_iso - 1, cz))
            cy = max(0, min(H_iso - 1, cy))
            cx = max(0, min(W_iso - 1, cx))

    # full-volume mask
    mask = np.zeros_like(vol, dtype=np.uint8)

    # no lesion -> all zeros
    if not has_lesion:
        info = {
            "label": 0,
            "center_zyx_iso": None,
            "start_zyx_iso": None,
            "size_vox_iso": None,
            "spacing_iso": tuple(map(float, spacing_zyx_iso)),
            "series_uid": str(series_uid),
            "raw_shape_guess": (raw_Z, raw_H, raw_W),
            "scale_raw_to_iso": (kz, ky, kx),
        }
        return mask, 0, info

    # ROI size in voxels (in iso space)
    Dz = max(1, int(round(size_mm[0] / tz)))
    Dy = max(1, int(round(size_mm[1] / ty)))
    Dx = max(1, int(round(size_mm[2] / tx)))

    # corner in iso coords
    z0, y0, x0 = _center_to_start(cz, cy, cx, Dz, Dy, Dx)

    # clamp ROI so it stays inside volume #THIS CAN BE A VULNERABILITY and get the coordinates of the ROI/positive part of mask)
    z_start = max(0, z0)
    y_start = max(0, y0)
    x_start = max(0, x0)
    z_end   = min(Z_iso, z0 + Dz)
    y_end   = min(H_iso, y0 + Dy)
    x_end   = min(W_iso, x0 + Dx)

    # set ROI region to 1
    mask[z_start:z_end, y_start:y_end, x_start:x_end] = 1

    info = {
        "label": 1,
        "center_zyx_iso": (int(cz), int(cy), int(cx)),
        "start_zyx_iso": (int(z_start), int(y_start), int(x_start)),
        "size_vox_iso": (int(Dz), int(Dy), int(Dx)),
        "spacing_iso": tuple(map(float, spacing_zyx_iso)),
        "series_uid": str(series_uid),
        "raw_shape_guess": (raw_Z, raw_H, raw_W),
        "scale_raw_to_iso": (kz, ky, kx),
    }

    return mask, 1, info #Last 2 for debugging and personal use


In [10]:
def center_crop_or_pad(vol, target_shape, z_start=0.6, x_start=0.5, y_start=0.5):
    """
    INPUT: 
    vol: Volume (full or mask)
    target_shape: Target shgpe of the volume and ROI
    start coordinate: id of the slice with lession, taken from the localizer's csv dataframe's SOPInstanceUID column
    
    OUPTUT:
    idx from the dicom slices list which has the lession or None if its a healthy patient and its not listed on localizers csv
    """
    vol_z, vol_y, vol_x = vol.shape
    tz, ty, tx = target_shape
    padz = max(tz - vol_z, 0)
    pady = max(ty - vol_y, 0)
    padx = max(tx - vol_x, 0)
    vol = np.pad(vol,
                 ((padz//2, padz - padz//2),
                  (pady//2, pady - pady//2),
                  (padx//2, padx - padx//2)),
                 mode='constant')
    z, y, x = vol.shape
    cz = int(round(z_start * (z - 1)))
    cy = int(round(y_start * (y - 1)))
    cx = int(round(x_start * (x - 1)))
    
    startz, starty, startx = cz-tz//2, cy-ty//2, cx-tx//2
    startz = max(0, min(z - tz, startz))
    starty = max(0, min(y - ty, starty))
    startx = max(0, min(x - tx, startx))
                       
    endz, endy, endx = startz + tz, starty + ty, startx + tx
    return vol[startz:endz, starty:endy, startx:endx]

def mask_validity(original_mask, resampled_mask):
    """
    mask: numpy array [D, H, W] after center_crop_or_pad
    Returns: (is_valid: bool, reason: str) True if the mask was not lost False if the mask was completely lost
    """
    # binarize in case it has been saved as float
    #m = mask > eps
    orig_sum = float(original_mask.sum())
    new_sum  = float(resampled_mask.sum())
    
    if orig_sum > 0 and new_sum == 0:
        print('mask lost')
        return False

    return True

# Data Loader

In [11]:
class Volume_mask_Dataset(Dataset):
    def __init__(self, train_df=train_df, series_root=DICOM_PATH, 
                 target_iso_full=TARGET_ISO_BRAIN, 
                 roi_mm=(60,60,60), out_shape=TARGET_SIZE, loc_df=loc_df):
        
        self.df = train_df.reset_index(drop=True)
        self.root = series_root
        self.target_iso_full = target_iso_full
        self.roi_mm = roi_mm
        self.out_shape = out_shape
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        sid = str(self.df[ID_COL].iloc[idx])
        dicom_dir = os.path.join(self.root, sid)
        
        ############ Load the raw DICOM volume ##########################
        modality, spacing_zyx_raw, sample_volume_raw, the_series_uid, dicom_slices = load_dicom_series(dicom_dir)


        # Get the raw volume before resampling
        sample_volume_raw = modality_specific_preprocessing(sample_volume_raw, modality, cta_startegy="hu_window")
        # Remove the leading frame dimension if it is 1, issue found while debugging
        if sample_volume_raw.ndim == 4 and sample_volume_raw.shape[0] == 1:
            sample_volume_raw = np.squeeze(sample_volume_raw, axis=0)

        ############# Resample to isotropic spacing #######################
        spacing_zyx_iso, volume_iso = resample_isotropic(sample_volume_raw, spacing_zyx_raw, target_iso=TARGET_ISO_BRAIN)
        ############### Generate full-volume mask (1 in ROI, 0 elsewhere) ###############
        original_mask, _, _ = lesion_mask_full_volume(volume_iso, 
                                                 dicom_slices, 
                                                 spacing_zyx_raw, 
                                                 spacing_zyx_iso, 
                                                 loc_df, 
                                                 series_uid=the_series_uid, 
                                                 size_mm=self.roi_mm)
        resampled_mask = center_crop_or_pad(original_mask, TARGET_SIZE)
        volume_iso = center_crop_or_pad(volume_iso, TARGET_SIZE)
        return sid, volume_iso, original_mask, resampled_mask

In [12]:
vol_dir = '/kaggle/working/volumes' #DIRECTORY TO SAVE PRE-PROCESSED VOLUMES
mask_dir = '/kaggle/working/masks' #DIRECTORY TO SAVE PRE-PROCESSED VOLUMES' MASKS

#EXCLUDES THOSE MASKS THAT GOT LOST WHEN STANDARDIZING VOLUMES TO SPECIFIC SIZE
def cache_dual_from_dataset(data: Volume_mask_Dataset, vol_dir=str, mask_dir=str):
    '''
    Apply pre-processing - standardize size - check if mask got lost - save volume and mask under same name but ddifferent folder.
    
    '''
    os.makedirs(vol_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    succesful = []
    unsuccesful = []

    for t in tqdm(range(5)): #For testing
    #for t in tqdm(range(len(data))): #For Running Full Data
        try:
            sid, x_full, original_mask, resampled_mask = ds[t]
            is_valid = mask_validity(original_mask, resampled_mask)
            if float(resampled_mask.sum()) > 0 and float(resampled_mask.sum()) >= 110592.0 * 0.8:
                unsuccesful.append(sid)
                print(f'Mask lost more than 20% of its volume standardizing vol size for: {sid}')
            if not is_valid:
                unsuccesful.append(sid)
                print(f'Mask lost when standardizing vol size for: {sid}')
                continue
                
            #print(is_valid)
            out_vols = os.path.join(vol_dir, f"{sid}.npz")
            out_mask = os.path.join(mask_dir, f"{sid}.npz")
            
            np.savez_compressed(out_vols, vol=x_full.astype(np.float16))
            np.savez_compressed(out_mask, vol=resampled_mask.astype(np.float16))
            succesful.append(sid)
        except Exception as e:
            print(e)
            unsuccesful.append(sid)
            
    print(f"Cached OK: {len(succesful)} | Failed: {len(unsuccesful)}")
    np.savez_compressed('/kaggle/working/bad.npz', lst=unsuccesful)
    if unsuccesful[:5]:
        print("Examples of failures:", unsuccesful[:5])
    return succesful, unsuccesful

In [13]:
#CHANGE AS APPLICABLE
quart =len(train_df)//4
train_df_quart1 = train_df.iloc[:quart]
# train_df_quart2 = train_df.iloc[quart:quart*2]
# train_df_quart3 = train_df.iloc[quart*2:quart*3]
# train_df_quart4 = train_df.iloc[quart*3:]

In [14]:
ds = Volume_mask_Dataset(train_df=train_df_quart1) #CHANGE as APPLICABLE
ok, bad = cache_dual_from_dataset(ds, vol_dir, mask_dir)

  0%|          | 0/5 [00:00<?, ?it/s]

Mask lost more than 20% of its volume standardizing vol size for: 1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317
Cached OK: 5 | Failed: 1
Examples of failures: ['1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317']


In [15]:
# Paths - CHANGE AS APPLICABLE
zip_path_masks = '/kaggle/working/masks_quart1.zip'
zip_path_vols = '/kaggle/working/vols_quart2.zip'
# Create zip
shutil.make_archive(base_name=zip_path_vols.replace('.zip', ''), format='zip', root_dir=vol_dir)
shutil.make_archive(base_name=zip_path_masks.replace('.zip', ''), format='zip', root_dir=mask_dir)

print(f"Zipped to: {zip_path_masks}")
print(f"Zipped to: {zip_path_vols}")

Zipped to: /kaggle/working/masks_quart1.zip
Zipped to: /kaggle/working/vols_quart2.zip


# Personal Use, Debugging Code

## Review Fails

In [16]:
# failed_patient= bad[1]
# train_df_quart2.loc[train_df['SeriesInstanceUID'] == failed_patient]

In [17]:
# good_patient= ok[18]
# train_df_quart2.loc[train_df['SeriesInstanceUID'] == good_patient]

In [18]:
# bad = np.load('/kaggle/working/bad.npz')["lst"]
# print(bad)

In [19]:
# roi_npz = np.load('/kaggle/working/masks/'+failed_patient +'.npz')["vol"]
# full_npz = np.load('/kaggle/working/volumes/'+failed_patient +'.npz')['vol']

# print(full_npz.shape)
# plt.imshow(full_npz[22])
# plt.title("Slice of 3d Volume (DICOM series)")
# plt.show()

# print(roi_npz.shape)
# print(np.where(roi_npz.max(axis=(1,2)) == 1)[0])
# print(len(np.where(roi_npz.max(axis=(1,2)) == 1)[0]))
# print(roi_npz.shape)
# plt.imshow(roi_npz[22])
# plt.title("Slice of 3d MASK (DICOM series)")
# plt.show()

# plt.imshow(roi_npz[22]*full_npz[22])
# plt.title("Slice of 3d ROI (DICOM series)")
# plt.show()

In [20]:
# roi_npz = np.load('/kaggle/working/masks/'+good_patient +'.npz')["vol"]
# full_npz = np.load('/kaggle/working/volumes/'+good_patient +'.npz')['vol']

# print(full_npz.shape)
# plt.imshow(full_npz[38])
# plt.title("Slice of 3d Volume (DICOM series)")
# plt.show()

# print(roi_npz.shape)
# print(np.where(roi_npz.max(axis=(1,2)) == 1)[0])
# print(len(np.where(roi_npz.max(axis=(1,2)) == 1)[0]))
# print(roi_npz.shape)
# plt.imshow(roi_npz[38])
# plt.title("Slice of 3d MASK (DICOM series)")
# plt.show()

# plt.imshow(roi_npz[38]*full_npz[38])
# plt.title("Slice of 3d ROI (DICOM series)")
# plt.show()

## Preview Files

In [21]:
# train_df_quart1.loc[9], train_df_quart1.loc[9]['SeriesInstanceUID']  #Check the data

In [22]:
# import os
# import glob
# folder = "/kaggle/working/masks"
# npz_files = sorted(glob.glob(os.path.join(folder, "*.npz")))

# print(f"Found {len(npz_files)} npz files")
# for i, f in enumerate(npz_files[:10]):  # show first 10 names
#     print(i, "->", os.path.basename(f))

# n = 9  # 0 = first file, 1 = second, etc.
# file_path = npz_files[n]
# print("Using file:", file_path)

# data = np.load(file_path)   # NpzFile object, behaves like a dict
# print("Keys in this npz:", data.files)

# mask = data[data.files[0]]   # grab the first array
# print("Image shape:", mask.shape)
# print("Image dtype:", mask.dtype)

In [23]:
# print(mask.shape)
# print(np.where(mask.max(axis=(1,2)) == 1)[0])
# print(len(np.where(mask.max(axis=(1,2)) == 1)[0]))
# plt.imshow(mask[80])
# plt.title("Slice of 3d Volume (DICOM series)")
# plt.show()

In [24]:
# folder = "/kaggle/working/volumes"
# npz_files = sorted(glob.glob(os.path.join(folder, "*.npz")))

# print(f"Found {len(npz_files)} npz files")
# for i, f in enumerate(npz_files[:10]):  # show first 10 names
#     print(i, "->", os.path.basename(f))

# n = 9  # 0 = first file, 1 = second, etc.
# file_path = npz_files[n]
# print("Using file:", file_path)

# data = np.load(file_path)   # NpzFile object, behaves like a dict
# print("Keys in this npz:", data.files)

# img = data[data.files[0]]   # grab the first array
# print("Image shape:", img.shape)
# print("Image dtype:", img.dtype)

In [25]:
# print(img.shape)
# plt.imshow(img[92])
# plt.title("Slice of 3d Volume (DICOM series)")
# plt.show()

In [26]:
# print(img.shape)
# plt.imshow(img[79]*mask[79])
# plt.title("Slice of 3d Volume (DICOM series)")
# plt.show()